# Running bash commands from your notebook

First, let's install all the dependencies. 

You can directly run bash commands in your notebook, by either prefixing your commands with an exclamation mark `!`:
```ipython
[1] !echo "this is a bash command"
this is a bash command

[2] !ls
/home/user/git_repos/FNO_workshop
```

or by starting your cell with the `%%bash` ipython magic. 

Let's see a simple example:

In [3]:
%%bash

for var in hello world
do
  echo ${var} 
done

hello
world


# Installing the dependencies

Now, let's install the dependencies.

In [4]:
%%bash 

target_folder='./temp'
[ -d ${target_folder} ] || mkdir -p ${target_folder}
cd temp

git clone https://github.com/tensorly/tensorly 
cd tensorly
python -m pip install -e .
cd ..

git clone https://github.com/tensorly/torch
cd torch
python -m pip install -e .
cd ..

git clone https://github.com/NeuralOperator/neuraloperator
cd neuraloperator
python -m pip install -e .

fatal: destination path 'tensorly' already exists and is not an empty directory.


Obtaining file:///home/workstation/code/bootcamp/temp/tensorly
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: tensorly
    Found existing installation: tensorly 0.8.1
    Uninstalling tensorly-0.8.1:
      Successfully uninstalled tensorly-0.8.1
  Running setup.py develop for tensorly


fatal: destination path 'torch' already exists and is not an empty directory.


Obtaining file:///home/workstation/code/bootcamp/temp/torch
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: tensorly-torch
    Found existing installation: tensorly-torch 0.4.0
    Uninstalling tensorly-torch-0.4.0:
      Successfully uninstalled tensorly-torch-0.4.0
  Running setup.py develop for tensorly-torch


fatal: destination path 'neuraloperator' already exists and is not an empty directory.


Obtaining file:///home/workstation/code/bootcamp/temp/neuraloperator
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: neuraloperator
    Found existing installation: neuraloperator 0.1.3
    Uninstalling neuraloperator-0.1.3:
      Successfully uninstalled neuraloperator-0.1.3
  Running setup.py develop for neuraloperator


In [5]:
!pip install gpustat
!pip install gdown
!pip install opt-einsum
!pip install h5py wandb ruamel.yaml zarr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.1 MB/s eta 0:00:00
  Created wheel for gpustat: filename=gpustat-1.0.0-py3-none-any.whl size=19867 sha256=0e141300a43ba54d293306f8dbd46050956fcb96074f5a949acd61bf896d03ae
  Stored in directory: /home/workstation/.cache/pip/wheels/1b/ed/14/0d513c962b25da841c42022cb5847c2ef835902c8563b8fb01
Successfully built gpustat
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.2/132.2 kB 1.8 MB/s eta 0:00:00a 0:00:01


# Prepare data 

In [6]:
import os
import requests
import hashlib
url_dict = {
    'darcyflow-1':'https://caltech-pde-data.s3.us-west-2.amazonaws.com/piececonst_r241_N1024_smooth1.mat', 
    'darcyflow-2': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/piececonst_r241_N1024_smooth2.mat', 
    'Navier-Stokes': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/ns_V1e-3_N5000_T50.mat', 
    'darcy-test-32': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/darcy_test_32.pt', 
    'darcy-test-64': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/darcy_test_64.pt', 
    'darcy-train-32': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/darcy_train_32.pt', 
    'darcy-train-64': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/darcy_train_64.pt', 
    'KF-Re100': 'https://caltech-pde-data.s3.us-west-2.amazonaws.com/KFvorticity_Re100_N50_T500.npy'
}

chksum_dict = {
    'piececonst_r241_N1024_smooth1.mat': '5ab3edf67bb5fd6d49ebf308cd79ed70340106d1a18af8a8439d3e7fc8e82d21', 
    'piececonst_r241_N1024_smooth2.mat': '51a818ed2e4f08752eea5d3f137f0e00271589c48297a46c641382a51eb80acf', 
    'ns_V1e-3_N5000_T50.mat': '78b8d9e83d767dc7050fb8145ee7e7f11e2d18d325bff9abc7f108ec3292ee78', 
    'darcy_train_64.pt': 'c05770239c91ebf093ea971e4d724008a49c9f21b5363fcf182e80499fae7fb4', 
    'darcy_train_32.pt': 'b8d8095d3832ed67f55b4a8fcb1970618b4ca2c6fc91aee2fe49b9c9b2c071ae', 
    'darcy_test_64.pt': '2220bb25c920109e9565a7fc07b675de16d124d563996f6e7256e2faa1fde24f', 
    'darcy_test_32.pt': '65137910193a553295c26e3d8273761daa44766597f4b34cfb12299fc6e3f311', 
    'KFvorticity_Re100_N50_T500.npy': '55f5af44a732a7843d631ace6384ac75c787d4fb36765b2e83ce1febb52d5463'
}

def download_file(url, file_path):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024 * 1024):
                f.write(chunk)
    print('Complete')


## Download Darcy datasets

In [7]:
data_root = 'data'
darcy_dir = os.path.join(data_root, 'darcy_flow')
os.makedirs(darcy_dir, exist_ok=True)

day1_data = ['darcy-train-64', 'darcy-train-32', 'darcy-test-64', 'darcy-test-32']

for key in day1_data:
    value = url_dict[key]
    print(f'Downloading {value}...')
    filename = os.path.basename(value)
    save_path = os.path.join(darcy_dir, filename)
    download_file(url=value, file_path=save_path)


Complete
Complete
Complete
Complete


In [8]:
# verify data integrity
for data_file in os.listdir(darcy_dir):
    data_path = os.path.join(darcy_dir, data_file)
    with open(data_path, 'rb') as f:
        data = f.read()
        sha256 = hashlib.sha256(data).hexdigest()
        if sha256 == chksum_dict[data_file]:
            print(f'{data_file} verified!')
        else:
            print(f'{data_file} verfication failed!')

darcy_test_32.pt verified!
darcy_train_64.pt verified!
darcy_train_32.pt verified!
darcy_test_64.pt verified!


### Download KF datasets (2d NS)

In [9]:
data_root = 'data'
kf_dir = os.path.join(data_root, 'kf')
os.makedirs(kf_dir, exist_ok=True)

kf_data = ['KF-Re100']
for key in kf_data:
    value = url_dict[key]
    print(f'Downloading {value} to {kf_dir}')
    filename = os.path.basename(value)
    save_path = os.path.join(kf_dir, filename)
    download_file(url=value, file_path=save_path)

Complete


In [10]:
for data_file in os.listdir(kf_dir):
    data_path = os.path.join(kf_dir, data_file)
    with open(data_path, 'rb') as f:
        data = f.read()
        sha256 = hashlib.sha256(data).hexdigest()
        if sha256 == chksum_dict[data_file]:
            print(f'{data_file} verified!')
        else:
            print(f'{data_file} verfication failed!')


KFvorticity_Re100_N50_T500.npy verified!
